In [76]:
import numpy as np
import pandas as pd
import jieba

In [77]:
#read weekly_popular.xlsx
weekly_popular = pd.read_excel('weekly_popular.xlsx')

In [89]:
def cut_words(column_name):
    column_cut = []
    #get column
    column = weekly_popular[column_name]
    for i in column:
        i = str(i)
        column_cut.append(jieba.lcut(i))
    wordlist = []
    for i in column_cut:
        for j in i:
            wordlist.append(j)
    #save word_list to word_list.csv
    wordlist = pd.DataFrame(wordlist)
    #drop nan
    wordlist = wordlist.dropna()
    wordlist.to_csv('{}_wordlist.csv'.format(column_name),index=False)
    #remove stop words from cn_stopwords.txt
    stopwords = [line.strip() for line in open('cn_stopwords.txt',encoding='UTF-8').readlines()]
    wordlist_removed = wordlist[~wordlist.isin(stopwords)]
    #drop nan
    wordlist_removed = wordlist_removed.dropna()
    #save word_list to word_list_noStopWords.csv
    wordlist_removed.to_csv('{}_wordlist_noStopWords.csv'.format(column_name),index=False)

In [92]:
cut_words('title')
cut_words('desc')
cut_words('dynamic')
cut_words('rcmd_reason')